<h1><center>Clustering analysis Google Analytics, Full explained with GridSearch, Neural Networks, RandomForest and Logistic Regression models.

Pleas, any question, doubt or suggestion I will be happy to answer you.

# ***Objetive:***

We work as data scientists for a retail company that, due to the change in customer consumption habits, is widely promoting the online sales service. The company wants to run a machine learning model to rank customers based on the likelihood of generating revenue when shopping on the web.

The goal is to perform a series of specific actions for customers who are most likely to make purchases on the web.

In [ ]:
#Import necesary libraries 
import pandas as pd # Basic tool for transform and clean Data. 
pd.options.display.max_columns = 100 # This option lets us see on the screen 100 columns of informatio
                                    # that is really usefull for a better visualitation of our data
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
import warnings
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')

In [ ]:
XY = pd.read_csv('../input/online-shoppers-intention/online_shoppers_intention.csv')

In [ ]:
print(u'- El número de filas en el dataset es: {}'.format(XY.shape[0]))
print(u'- El número de columnas en el dataset es: {}'.format(XY.shape[1]))
print(u'- Los nombres de las variables son: {}'.format(list(XY.columns)))
XY[:2]

In [ ]:
XY.info()

In [ ]:
XY.describe(include = 'bool')

At this point we coud see that we need a binary cluster Ml model to indetify what kind of user will buy in the online stores. So, we will start  
# <center> **Preprocessing Data**

In [ ]:
#We just have few NAN in our dataset so we'll just delete them.  
XY.isnull().sum()

In [ ]:
XY = XY.dropna()
XY.isnull().sum()
#Now we dont have nan.

## <center> Our next important step is analyze column by column, looking for stranger values

In [ ]:
#Function for describe every column:

def describe_column(df, col):
    print(f'Columna: {col}  -  Tipo de datos: {df[col].dtype}')
    print(f'Número de valores nulos: {df[col].isnull().sum()}  -  Número de valores distintos: {df[col].nunique()}')
    print('Valores más frecuentes:')
    for i, v in df[col].value_counts().iloc[:20].items() :
        print(i, '\t', v)

In [ ]:
describe_column(XY, 'Administrative')
#All looks good.

In [ ]:
describe_column(XY, 'Administrative_Duration')
# Since this columns tell us the time spent on certain type of page, a -1 value
# its a strange value, then we can infer that '-1' represent nan values also.  
# we will take a look.

In [ ]:
menos_1 = XY['Administrative_Duration'] == -1
menos_uno = XY[menos_1]

In [ ]:
menos_uno.head(3)

All this '-1' looks like wrong values so we'll drop them. We can do it without problems, because there are just few values and looks like there are conected with other -1 values in the nexts columns.

In [ ]:
XY = XY.drop(XY[XY['Administrative_Duration']==-1].index)

In [ ]:
#Now our columns looks better. 
describe_column(XY, 'Administrative_Duration')

In [ ]:
describe_column(XY, 'Informational') #All perfect. Next one

In [ ]:
describe_column(XY, 'Informational_Duration') #All perfect. Next one

In [ ]:
describe_column(XY, 'ProductRelated') #All perfect. Next one

In [ ]:
describe_column(XY, 'ProductRelated_Duration') #All perfect. Next one

In [ ]:
describe_column(XY, 'BounceRates') #All perfect. Next one

In [ ]:
describe_column(XY, 'ExitRates') #All perfect. Next one

In [ ]:
describe_column(XY, 'PageValues') #All perfect. Next one

In [ ]:
describe_column(XY, 'SpecialDay') #All perfect. Next one

In [ ]:
describe_column(XY, 'Month') #We have two months left. 

In [ ]:
describe_column(XY, 'OperatingSystems') #Ok

In [ ]:
describe_column(XY, 'Browser') #Ok

In [ ]:
describe_column(XY, 'Region') #Ok

In [ ]:
describe_column(XY, 'TrafficType') #Ok

In [ ]:
describe_column(XY, 'VisitorType') #This 'other' values looks suspicius. But ill keep
#them for the moment. 

In [ ]:
describe_column(XY, 'Weekend')

In [ ]:
describe_column(XY, 'Revenue')

### For graphs our variables, the first step is transform the categorical values to numeric.

In [ ]:
# Transform the booleans object to numeric for our future ML or DL model and graphs
XY['Weekend'] = XY['Weekend'].map({False:0, True:1})
XY['Revenue'] = XY['Revenue'].map({False:0, True:1})

### For take a desicion about wich method we'll use, first we need to know how many variables have 'Month' and 'VisitorType' colums.

In [ ]:
# 10 categories (Months)
XY.Month.unique() 

In [ ]:
# Just 3 categories
XY.VisitorType.unique()

### Since, we just have 13 categories, use the dummies functions its a good options for our dataframe. In case of have so much more categories values probably we should choose another option.
Mainly because the Dummies function adds X amount of columns to our dataframe based on how many categories our base columns have.

In [ ]:
X_D = pd.get_dummies(XY,columns=['Month','VisitorType'],drop_first=True)
X_D[:3]

In [ ]:
X_numeric = XY.drop(columns=['Month', 'VisitorType', 'Revenue'])
# We create this DF witouth categories columns for plot

#### <center> We finish, now we'll separete our target Y from the features X 

In [ ]:
X = X_D.drop('Revenue', axis=1)
y = X_D['Revenue']

## <center> GRAPHS

In [ ]:
plt.figure(figsize=(15,7))
ax = sns.boxplot(data=X)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.title(u'Representación de cajas de las variables independientes X')
plt.ylabel('Valor de la variable normalizada')
_ = plt.xlabel('Nombre de la variable')

All the variables are concentrated on the lower part of the graph, with the expeption of 'ProductRelated_Duration'. That is a really good reason for standarize the Data before to train our ML or DL model.

**its Important** also, that big outlier in 'ProductRelated_Duration', but taking in consideretion that this variable is the time spent inside of the specefic shoping page probably is not a wrong value. 

<center> <h1> Outlier

In [ ]:
# Detect the outliers on ProductRelated_Duration, which, according to our graphs,
# was the only  column with possible outliers

q = X["ProductRelated_Duration"].quantile(0.99)

# and then filter with:

time_spent=X[X["ProductRelated_Duration"] > q]

In [ ]:
time_spent_in_seconds = time_spent['ProductRelated_Duration'].mean()

In [ ]:
time_spent_in_hours = time_spent_in_seconds/3600 # This columns its in seconds and here we transform it
# in hours for better analysis.

In [ ]:
round(time_spent_in_hours, 2)

Here we coud see that the mean of spent time in the shop page for the 0.01 higher quantile of 'ProductRelated_Duration' columns is 3.59 hours. That looks really normal. 

¿Who hasnt spent 3 hours choosing the best product?

**The otliers will remain in our dataframe**

<center><h1> Graphs

In [ ]:
# Functions that we'll use. Its not really necessary at this point that you understand every line of
# code.

def relaciones_vs_target(X, Y, return_type='axes'):
    '''
    Función que representa gráficos de dispersión de las variables
    en X en función a la variable Y
    '''
    fig_tot = (len(X_numeric.columns))
    fig_por_fila = 4.
    tamanio_fig = 4.
    num_filas = int( np.ceil(fig_tot/fig_por_fila) )    
    plt.figure( figsize=( fig_por_fila*tamanio_fig+5, num_filas*tamanio_fig+5 ) )
    c = 0 
    for i, col in enumerate(X.columns):
        plt.subplot(num_filas, fig_por_fila, i+1)
        sns.scatterplot(x=X_numeric[col], y=Y)
        plt.title( '%s vs %s' % (col, 'target') )
        plt.ylabel('Target')
        plt.xlabel(col)
    plt.show()

def represento_doble_hist(x_1, x_0, n_bins=11, title='', label_1='Clase 1', 
                          label_0='Clase 0', density=0):
    '''
    Función que recibe dos distribuciones de probabilidad y las representa
    en el mismo gráfico
    '''
    bins = n_bins
    plt.hist(x_1, bins, density = density, alpha=0.5, label=label_1, color='red')    
    plt.hist(x_0, bins, density = density, alpha=0.5, label=label_0, color='green')
    plt.title(title)
    plt.legend(loc='best') 

def hist_pos_neg_feat(x, y, density=0, nbins=11, targets=(0,1)):
    '''
    Representa las variables en x divididas en dos distribuciones
    según su valor de y sea 1 o 0
    '''
    fig_tot = len(x.columns)
    fig_tot_fila = 3.; fig_tamanio = 5.
    num_filas = int( np.ceil(fig_tot/fig_tot_fila) )
    plt.figure( figsize=( fig_tot_fila*fig_tamanio+2, num_filas*fig_tamanio+2 ) )
    target_neg, target_pos = targets
    for i, feat in enumerate(x.columns):
        plt.subplot(num_filas, fig_tot_fila, i+1);
        plt.title('%s' % feat)
        idx_pos = y == target_pos
        idx_neg= y == target_neg
        represento_doble_hist(x[feat][idx_pos].values, x[feat][idx_neg].values, nbins, 
                   density = density, title=('%s' % feat))

In [ ]:
hist_pos_neg_feat(X,y)

According to these graphs, we can see that customers who complete online purchases are evenly distributed within our data. Only with a visual analysis can not obtain new value data to contribute to the Marketing department.

In [ ]:
matriz_correlaciones = X_numeric.corr(method='pearson')
n_ticks = len(X_numeric.columns)
plt.figure( figsize=(9, 9) )
plt.xticks(range(n_ticks), X_numeric.columns, rotation='vertical')
plt.yticks(range(n_ticks), X_numeric.columns)
plt.colorbar(plt.imshow(matriz_correlaciones, interpolation='nearest', 
                            vmin=-1., vmax=1., 
                            cmap=plt.get_cmap('Blues')))
_ = plt.title('Matriz de correlaciones de Pearson')

In [ ]:
# Correlation with Revenue
data_corr = XY.corr()['Revenue'] 
sns.barplot(data_corr[0:-1].index,data_corr[0:-1].values).set_title('Correlation with the Revenue')
plt.xticks(rotation = 90)
plt.show()

###  Resultantly, Page Values has the highest correlation( around 0.5) with Revenue compare to all other features.

In [ ]:
# Lets see the Ratio of Revenue in each types

plt.style.use('fivethirtyeight')
fig,ax = plt.subplots(nrows = 2, ncols = 4,figsize = (17,10))
fig.tight_layout(pad = 3)


adm_rev = XY[['Administrative','Revenue']]
rev_p1 = pd.DataFrame(XY.groupby('Revenue')['Administrative'].sum()).T
rev_p1.plot.bar(stacked=True,ax=ax[0,0])
ax[0,0].set_xticklabels(['Administrative'], rotation=360)
plt.legend(loc='best')


info_rev = XY[['Informational','Revenue']]
rev_p2 = pd.DataFrame(XY.groupby('Revenue')['Informational'].sum()).T
rev_p2.plot.bar(stacked=True,ax = ax[0,1],color = ['black','white'])
ax[0,1].set_xticklabels(['Informational'], rotation=360)
plt.legend(loc='best')

info_rev = XY[['ProductRelated','Revenue']]
rev_p2 = pd.DataFrame(XY.groupby('Revenue')['ProductRelated'].sum()).T
rev_p2.plot.bar(stacked=True,ax = ax[0,2],color = ['purple','red'])
ax[0,2].set_xticklabels(['ProductRelated'], rotation=360)
plt.legend(loc='best')



info_rev = XY[['OperatingSystems','Revenue']]
rev_p2 = pd.DataFrame(XY.groupby('Revenue')['OperatingSystems'].sum()).T
rev_p2.plot.bar(stacked=True,ax = ax[0,3],color = ['green','yellow'])
ax[0,3].set_xticklabels(['OperatingSystems'], rotation=360)


info_rev = XY[['Browser','Revenue']]
rev_p2 = pd.DataFrame(XY.groupby('Revenue')['Browser'].sum()).T
rev_p2.plot.bar(stacked=True,ax = ax[1,0],color = ['navy','orange'])
ax[1,0].set_xticklabels(['Browser'], rotation=360)


info_rev = XY[['Region','Revenue']]
rev_p2 = pd.DataFrame(XY.groupby('Revenue')['Region'].sum()).T
rev_p2.plot.bar(stacked=True,ax = ax[1,1],color = ['black','red'])
ax[1,1].set_xticklabels(['Region'], rotation=360)


info_rev = XY[['TrafficType','Revenue']]
rev_p2 = pd.DataFrame(XY.groupby('Revenue')['TrafficType'].sum()).T
rev_p2.plot.bar(stacked=True,ax = ax[1,2],color = ['blue','green'])
ax[1,2].set_xticklabels(['TrafficType'], rotation=360)

fig.delaxes(ax[1,3])

Columns :'Administrative', 'Informational' and 'ProductRelated' there are the most related with the revenue results. We have to keept that in mind. 

In [ ]:
#All perfect. Continue. 

<center><h1> Standardization of data:

In [ ]:
obj_escalar = StandardScaler()
X_estandarizado = obj_escalar.fit_transform(X_D)
#Al estandarizar los datos mi regresion logistica me entraga un ROC de 1.

In [ ]:
X_estandarizado

In [ ]:
# Now, we divide our df in train and test for start applyng models

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


<center><H3> Now, We'll train 3 differents ML model looking for the best one 

<center><H2> Logistic Regresion

GridSearchCV its a extremly useful code for ours ML model. Its lets us find the best combination of parameters. We shoud use it always

In [ ]:
modelo1 = LogisticRegression()
parametros = {"C": [0.001, 0.008, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06], 
              "class_weight":['balanced', None]}

In [ ]:
modelo_gs = GridSearchCV(modelo1, param_grid=parametros,
                         cv = 5, scoring='roc_auc')
modelo_gs.fit(X_train, Y_train)

In [ ]:
print(modelo_gs.best_params_, "\nROC AUC: {}".format(round(modelo_gs.best_score_,2)))

0.88 ROC AUC its a really nice result. But we must continue looking for other models. 
Important that in the code cell above, 'best_params_ give us the result of the best parameters for use finally in our model for this Dataset

In [ ]:
df_search = pd.DataFrame.from_dict(modelo_gs.cv_results_)

In [ ]:
plt.xlabel('C')
plt.ylabel('roc_auc')
_ = plt.plot( df_search['param_C'], df_search['mean_test_score'], '.')

Analizando el modelo con el mejor alpha
En este paso nos quedamos con los mejores parámetros obtenidos en el paso anterior:

In [ ]:
reg_log =  LogisticRegression(C=modelo_gs.best_params_['C'],
                              class_weight=modelo_gs.best_params_['class_weight'])

Finally we train our model with the best parameters that GridSearch give us.

In [ ]:
reg_log.fit(X_train, Y_train)

Here, is when the Test Set comes into play. When you want to validate an already chosen and optimized model.
With that model optimized, I predict tests to see how it behaves on data that you haven't seen before

In [ ]:
y_test_pred_prob = reg_log.predict_proba(X_test)
y_test_pred_prob_pos = y_test_pred_prob[np.where(Y_test == 1)[0]]
y_test_pred_prob_neg = y_test_pred_prob[np.where(Y_test == 0)[0]]

In [ ]:
preds = y_test_pred_prob[:,1]
fpr, tpr, threshold = metrics.roc_curve(Y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.figure(figsize=(10,7))
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

This figure represents in green the probabilities assigned by the model to data that are 0s (the closer the green distribution is to 0, the better) and in red the probabilities assigned to the data that are 1s (the closer the red distribution is to 1 best).

Esta figura representa en verde las probabilidades que asigna el modelo a los datos que son 0s (cuanto más cerca de 0 la distribución verde mejor) y en rojo las probabilidades asignadas a los datos que son 1s (cuanto más cerca esté de 1 la distribución roja mejor).

In [ ]:
represento_doble_hist(y_test_pred_prob_pos[:, 1], y_test_pred_prob_neg[:, 1], n_bins=21, density=0)

We have to choose the umbral for separate our results which are continuous values into a binary option of Yes or Not (1 or 0) about the revenue probability.

In [ ]:
umbral = 0.6
y_umbralizadas = 1*(y_test_pred_prob[:, 1] > umbral)

In [ ]:
print(u"Matriz de confusión\n", metrics.confusion_matrix(Y_test, y_umbralizadas))
print("\nAccuracy\t{}".format(round(metrics.accuracy_score(Y_test, y_umbralizadas),2)))  
print("Sensitividad\t{}".format(round(metrics.recall_score(Y_test, y_umbralizadas),2)))
print(u"Precisión\t{}".format(round(metrics.precision_score(Y_test, y_umbralizadas),2)))    

In [ ]:
umbral = 0.7
y_umbralizadas = 1*(y_test_pred_prob[:, 1] > umbral)

In [ ]:
print(u"Matriz de confusión\n", metrics.confusion_matrix(Y_test, y_umbralizadas))
print("\nAccuracy\t{}".format(round(metrics.accuracy_score(Y_test, y_umbralizadas),2)))  
print("Sensitividad\t{}".format(round(metrics.recall_score(Y_test, y_umbralizadas),2)))
print(u"Precisión\t{}".format(round(metrics.precision_score(Y_test, y_umbralizadas),2)))    

We finally keep in 0.7 the threshold because it give us best performance. 

<center><h1> Neural Network Clasification

In [ ]:
modelo = MLPClassifier()
parametros = {'solver': ['lbfgs'], 
              'max_iter': [300,500, 800, 1000], # Iteraciones máximas en cada red
              'alpha': 10.0 ** -np.arange(0.5, 2), # Parámetro de regularización L2 para evitar sobreajuste
              'hidden_layer_sizes':np.arange(10, 35), # Número de neuronas en cada capa
              'random_state':[0]}

In [ ]:
modelo_gs2 = GridSearchCV(modelo, param_grid=parametros, cv = 3, 
                         scoring='roc_auc', n_jobs=-1, verbose=10)
modelo_gs2.fit(X_train, Y_train)

In [ ]:
#In this case for GridSearch the Max Iter was 500, and we obtein the result that this param was the
#Best, so now ill adjust GridSearch for testing with Higher Max Iter and looks if our model improve.
print(modelo_gs2.best_params_, "\nROC AUC: {}".format(round(modelo_gs2.best_score_,2)))

In [ ]:
#How we coud see, the ROC AUC improve using the max Iter again that i gave. First ill check with the
# test DF for look if we also get the same improve or in the worst case we get and overfit. 
print(modelo_gs2.best_params_, "\nROC AUC: {}".format(round(modelo_gs2.best_score_,2)))

## Best parameters

As in the other models, we obtain the best parameters found and fit a model with those parameters:

In [ ]:
mejor_modelo = MLPClassifier(**modelo_gs2.best_params_, verbose=10)

In [ ]:
mejor_modelo.fit(X_train, Y_train)

In [ ]:
y_test_pred_prob = mejor_modelo.predict_proba(X_test) 
y_test_pred_prob_pos = y_test_pred_prob[np.where(Y_test == 1)[0]]
y_test_pred_prob_neg = y_test_pred_prob[np.where(Y_test == 0)[0]]

In [ ]:
preds = y_test_pred_prob[:,1]
fpr, tpr, threshold = metrics.roc_curve(Y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.figure(figsize=(10,7))
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
represento_doble_hist(y_test_pred_prob_pos[:, 1], y_test_pred_prob_neg[:, 1], n_bins=21, density=0)

In [ ]:
umbral = 0.5
y_umbralizadas = 1*(y_test_pred_prob[:, 1] > umbral)

In [ ]:
print(u"Matriz de confusión\n", metrics.confusion_matrix(Y_test, y_umbralizadas))
print("\nAccuracy\t{}".format(round(metrics.accuracy_score(Y_test, y_umbralizadas),2)))  
print("Sensitividad\t{}".format(round(metrics.recall_score(Y_test, y_umbralizadas),2)))
print(u"Precisión\t{}".format(round(metrics.precision_score(Y_test, y_umbralizadas),2)))  

In [ ]:
# Modifing the Maxiter from 500 to 1000, improve the model but only on the train df. it keept
# 0.88 accurary. Probably we coud tried with a higher number of itermax.

<center><h1> RandomForestClassifier

In [ ]:
rfclf = RandomForestClassifier(n_estimators = 30,max_depth = 10,random_state = 101)


rfclf.fit(X_train,Y_train)
pred = rfclf.predict(X_test)
print(classification_report(Y_test,pred))

In [ ]:
# Lets Optimize the Random Forest Classifier using GridSearch
param_grid = {
    'n_estimators' : [80,100],
    'max_depth' : [10,15],
    'min_samples_leaf' : [2,3],
    'min_samples_split': [2,4]
}

gridsearch = GridSearchCV(estimator=rfclf,param_grid=param_grid,verbose = 1)
gridsearch.fit(X_train,Y_train)

In [ ]:
gridsearch.best_params_

In [ ]:
rfclf = RandomForestClassifier(n_estimators = 100,max_depth = 10,min_samples_leaf = 3, min_samples_split = 2,random_state = 101)
rfclf.fit(X_train,Y_train)
pred = rfclf.predict(X_test)
print(classification_report(Y_test,pred))

# 0 is False, 1 is True, the precision of detecting True has increased
from sklearn.metrics import accuracy_score
rfacc = accuracy_score(Y_test,pred)

Finally we have developed the 3 ML models to predict whether a visitor to the e-commerce page will make a purchase or not. But as far as we have seen so far we have the big problem that these results are not easily transformed into value for the marketing department or someone else within the company.

This is due to the "Black Box" characteristic of the vast majority of the models used, which translates into not being able to extract important insights from the application of Machine Learning models.

It is because of the above that the "Permutation Importance" technique will finally be used to determine which of all our features are the ones that provide the closest approximation to determining whether the purchase will be made or not.

-----------

Finalmente hemos desarrollado los 3 modelos de ML para predecir si un visitante a la pagina de comercio electrónico realizara una compra o no. Pero hasta donde hemos visto hasta ahora nos surge el gran problema de que estos resultados no son facilmente transformables en valor para el departamento de marketing o algun otro dentro de la compañia. 

Esto es debido a la caracteristica de "Caja Negra" de la gran mayoria de los modelos utilizados, lo que se traduce en no poder extraer importantes insigths desde la aplicacion de modelos de Machine Learning.

Es en razon a lo anterior que se utilziara finalmente la tecnica de "Permutation Importance" para determinar cual de todas nuestras features son las que entregan mayor aproximacion a determinar si se realizará la compra o no. 

<h1><center> "Permutation Importance"

### Looking for the explanation of ours models. 

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rfclf, random_state=1).fit(X_test, Y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

<center><h1> Shap method

We see the same results again regarding which features are more important
____
Volvemos a ver los mismo resultados respecto a que featurees son mas importantes

From this application of 'PermutationImportance' to two of our models 'PageValues' appears to us as the main feature of importance when defining who will or will not make the purchase. Faced with this, our next step, in order to generate insights and transform our analysis into money for the company, is to thoroughly analyze that column and how it behaves.
This could only be done by means of very detailed and advanced statistical graphics, which is not objective in this work, but it is undoubtedly what would make the difference between a successful case or just one more Machine Learning model.
___
Desde esta aplicacion de 'PermutationImportance' a dos de nuestros modelos 'PageValues' nos aparece como la principal feature de importancia al momento de definir quien realizará o no la compra. Frente a esto nuestro siguiente paso, de vista a generar insights y tranformar nuestro analisis en dinero para la empresa es analisar a fondo aquella columna y como se comporta. 
Esto solo se podria realizar mediantes graficas estadisticas muy detalladas y avanzadas, lo cual no es objetivo en el presente trabajo, pero sin duda es lo que marcaría la diferencia entre un caso de exito o solo un modelo más de Machine Learning.

<center><h1> Manual Insigths
   

In [ ]:
compradores_si = XY['Revenue'] == 1
compradores = XY[compradores_si]

In [ ]:
compradores_no = XY['Revenue'] == 0
no_compradores = XY[compradores_no]

In [ ]:
for i,k in zip(compradores.mean(),no_compradores.mean()):
    print (i,k)

In [ ]:
print(compradores.mean(), ), print(no_compradores.mean())

<center><h1> Conclusion

From my analysis, customers who are interested on buying the products :
Tend to stay longer on the website especially when they are on the website which is productrelated. Less likely to bounce or exit

All of the aforementioned would not charge an important value for the company if this model is not put into production in the cloud to analyze the data of potential buyers in real time and in this way direct marketing strategies to all those people who they are more likely to make a purchase.
___
Finalmente, como todo lo obtenido mediante el presente analisis, se puede corroborar las caracteristicas principales de los compradores quienes en promedio gastan mas tiempo visitando las paginas web, como tambien el numero de veces que las visitan, al mismo tiempo que los promedios de BounceRate y ExitRates son menores para estos grupos.

Todo lo anteriomente expuesto no cobraría un valor importante para la compañia si es que este modelo no se pone en produccion en la nube para analizar en tiempo real los datos de los posibles compradores y de esta forma direccionar las estrategias de marketing a todos aquellas personas que se encuentran mas propensas a realizar una compra. 

In [ ]:
final_plot= XY.drop(columns= ['OperatingSystems', 'Browser', 'Region', 'TrafficType'])
final_plot.groupby('Revenue').mean().plot(kind='bar', figsize=(15,7))
plt.title('Gasto medio por producto en cada clúster')
plt.xlabel(u'Número de clúster')
_ = plt.ylabel('Valor medio de gasto')